# 1. 기술 담당자에게 구매 결재 검토 요청을 전달해야 하는가 (구매 주문 승인)
<img src="20230520_224848.png" alt="image" width="600" height="375"><br>

## 캐런의 검토 요청 과정
캐런과 업무를 논의한 결과 그녀는 일반적으로 IT 제품과 유사한 제품의 구매 요청은 기술담당자에게 보냅니다.
</br>마우스나 키보드 같이 IT 제품에 연결해서 사용하는 소비제품은 기술 담당자에게 요청을 보내지 않습니다.</br> 그리고 구매 요청이 IT 부서에서 온 경우에도 굳이 기술 담당자에게 요청을 보내지 않습니다.

1. 캐런은 기술 담당자에게 결재안을 검토 요청 보내야 할까요, 아니면 재정 담당자에게 바로 보내도 될까요?
2. 기술 담당자는 결재안을 검토 한 후 승인하고 재정 담당자에게 결재안을 보내야 할까요, 아니면 바로 승인을 거부해야 할까요?
3. 기술 담당자가 승인한 결재안을 전달받은 재정 담당자가 이것을 검토한 후 승인하여 공급업체에 주문을 해야 할까요, 아니면 결재안을 반려하여 결재를 요청한 직원에게
다시 돌려보내야 할까요?

## Part 1: 데이터를 불러오고 살펴보기

In [41]:
# 필요한 라이브러리 불러오기
import sys
import pandas as pd
from time import sleep
from sklearn.model_selection import train_test_split

In [42]:
df = pd.read_csv('./orders_with_predicted_value.csv')
df.info()
# tech_approval_required : 기술담당자에게 결재안 검토 요청을 보낸다면 1, 아니면 0
# requester_id :구매 요청을 한 직원 ID
# role : 구매요청을 한 직원이 IT 부서소속이면 tech, 아니면 non-tech
# product : 구매 요청을 한 제품
# quantity : 수량
# price : 가격
# total : 수량 X 가격
print("===================================")
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   tech_approval_required  1000 non-null   int64 
 1   requester_id            1000 non-null   object
 2   role                    1000 non-null   object
 3   product                 1000 non-null   object
 4   quantity                1000 non-null   int64 
 5   price                   1000 non-null   int64 
 6   total                   1000 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 54.8+ KB


,tech_approval_required,requester_id,role,product,quantity,price,total
0,0,E2300,tech,Desk,1,664,664
1,0,E2300,tech,Keyboard,9,649,5841
2,0,E2374,non-tech,Keyboard,1,821,821
3,1,E2374,non-tech,Desktop Computer,24,655,15720
4,0,E2327,non-tech,Desk,1,758,758


In [43]:
print(f'데이터 총 행수: {df.shape[0]}')
# 193개의 주문은 기술 담당자에게 보냈고, 807개의 주문은 기술 담당자에게 보내지 않음
print(df[df.columns[0]].value_counts())

데이터 총 행수: 1000
0    807
1    193
Name: tech_approval_required, dtype: int64


## Part 2: 데이터를 모델에 적합한 형태로 가공

In [44]:
# 범주형 변수를 인코딩함
encoded_data = pd.get_dummies(df)
encoded_data.head()

,tech_approval_required,quantity,price,total,requester_id_E2300,requester_id_E2301,requester_id_E2302,requester_id_E2303,requester_id_E2304,requester_id_E2306,...,requester_id_E2400,role_non-tech,role_tech,product_Chair,product_Cleaning,product_Desk,product_Desktop Computer,product_Keyboard,product_Laptop Computer,product_Mouse
0,0,1,664,664,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,0,9,649,5841,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,0,1,821,821,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,1,24,655,15720,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,0,1,758,758,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [45]:
# 상관관계 확인 후 상관계수 절댓값이 0.1이상인 열 선택
# 하필 0.1이상으로 하는 이유? -> 머신러닝의 성능 향상에 도움이 되지 않는 노이즈 성분을 제거하고, 데이터를 의미있게 정리할 수 있게함.
corrs = encoded_data.corr()['tech_approval_required'].abs()
columns = corrs[corrs > .1].index
corrs = corrs.filter(columns)
corrs

tech_approval_required      1.000000
role_non-tech               0.122454
role_tech                   0.122454
product_Chair               0.134168
product_Cleaning            0.191539
product_Desk                0.292137
product_Desktop Computer    0.752144
product_Keyboard            0.242224
product_Laptop Computer     0.516693
product_Mouse               0.190708
Name: tech_approval_required, dtype: float64

In [46]:
# 상관관계 절댓값이 0.1이상인 테이블 출력
encoded_data = encoded_data[columns]
encoded_data.head()

,tech_approval_required,role_non-tech,role_tech,product_Chair,product_Cleaning,product_Desk,product_Desktop Computer,product_Keyboard,product_Laptop Computer,product_Mouse
0,0,0,1,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,0,0,0,1,0,0,0
4,0,1,0,0,0,1,0,0,0,0


## Part 3: 학습용, 검증용, 테스트용 데이터셋 생성

In [47]:
# 학습용 데이터 70%, 검증용 데이터 20%, 테스트용 데이터 10% 설정
train_df, val_and_test_data = train_test_split(encoded_data, test_size=0.3, random_state=0)
val_df, test_df = train_test_split(val_and_test_data, test_size=0.333, random_state=0)

In [48]:
X_train=train_df.iloc[:,1:]
y_train=train_df['tech_approval_required']

X_val=val_df.iloc[:,1:]
y_val=val_df['tech_approval_required']

X_test=test_df.iloc[:,1:]
y_test=test_df['tech_approval_required']

## Part 4: 머신러닝 모델 학습

In [49]:
# XGBoost 불러오기
import xgboost as xgb 
# XGBoost 분류 모델 하이퍼파라미터 설정
params = {
    'max_depth': 5,
    'subsample':0.7,
    'objective': 'binary:logistic',
    'eval_metric' : 'auc',
    'num_rounds' : 100,
    'early_stopping_rounds':10
}


# 학습,검증,테스트 데이터를 DMatrix로 변환
dtrain = xgb.DMatrix(data=X_train, label = y_train)
dval = xgb.DMatrix(data=X_val, label=y_val)
dtest = xgb.DMatrix(data=X_test, label=y_test)

# 모델을 학습합니다.
wlist = [(dtrain, 'train'), (dtest,'eval')]
# 하이퍼 파라미터와 early stopping 파라미터를 train() 함수의 파라미터로 전달
xgb_model = xgb.train(params = params, dtrain=dtrain, evals=wlist)

[02:32:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "early_stopping_rounds", "num_rounds" } are not used.

[0]	train-auc:0.99973	eval-auc:0.99709
[1]	train-auc:0.99973	eval-auc:0.99709
[2]	train-auc:0.99973	eval-auc:0.99709
[3]	train-auc:0.99973	eval-auc:0.99709
[4]	train-auc:0.99973	eval-auc:0.99709
[5]	train-auc:0.99973	eval-auc:0.99709
[6]	train-auc:0.99973	eval-auc:0.99709
[7]	train-auc:0.99973	eval-auc:0.99709
[8]	train-auc:0.99973	eval-auc:0.99709
[9]	train-auc:0.99973	eval-auc:0.99709


## Part 5: 모델 테스트

In [50]:
# 테스트 데이터 예측
preds = xgb_model.predict(dtest)
preds = [ 1 if x > 0.5 else 0 for x in preds]

In [51]:
# 혼동행렬, 정확도, 정밀도, 재현율, F1, AUC 불러오기
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
    
get_clf_eval(y_test, preds)

오차행렬:
 [[85  1]
 [ 0 14]]

정확도: 0.9900
정밀도: 0.9333
재현율: 1.0000
F1: 0.9655
AUC: 0.9942


## Part 6: 요약
- 회사 업무에서 의사결정의 지점을 파악함으로써 머신러닝을 적용할 수 있는 기회를 파악할 수 있다.
- 어떤 규칙을 작성하지 않고도, 머신러닝 시스템에 1000개의 주문 기록과 각 주문마다 기술 담당자에게 구매 의사를 통보할 것인지 여부를 체크하여 데이터를 입력하기만 하면 머신러닝 시스템은 1000개의 데이터의 패턴을 판단한 후 앞으로 주문을 신규 추가 할 때 해당 결재요청이 기술 담당자에게 구매 결재 검토 요청을 전달해야 할 결재인지 아닌지 판단할 수 있음.
- 기술 담당자에게 구매 결재 검토 요청을 전달할지 여부를 결정하는 의사결정 시스템을 완성시킴